In [1]:
import pyspark

In [2]:
pyspark.__version__

'3.4.1'

In [3]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

## 1.  세션생성하기 

## 스파크(Spark)에서 세션(Session)

- 스파크 클러스터와의 상호 작용을 관리하는 데 사용되는 중요한 개념 중 하나.
- 스파크 세션은 스파크 애플리케이션을 실행하고 클러스터와 통신하는 데 필요한 정보와 상태를 저장하는 객체

### 스파크 세션이 하는 일 

- 클러스터 연결: 스파크 세션은 클러스터와 통신하기 위한 연결을 설정하고 관리합니다.

- 애플리케이션 설정: 스파크 애플리케이션의 구성 설정을 관리합니다. 이는 실행 모드, 메모리 설정, CPU 설정 등과 관련이 있습니다.

- 데이터 처리: 데이터를 읽고 처리하는 데 필요한 API 및 기능을 제공합니다. 예를 들어, 데이터프레임 API, 스트리밍 API 등을 사용할 수 있습니다.

- 작업 모니터링: 스파크 애플리케이션의 진행 상황 및 로그를 모니터링합니다




### 스파크 세션 클래스 사용하기 

In [4]:
from pyspark.sql import SparkSession

### 스파크 세션 빌더하기

- 주요 컨피그를 세팅 : 현재는 로컬로 처리
- 스파크 호스트 등은 실제 스파크를 설치 서버의 주소 

In [5]:
spark = (SparkSession.builder
           .master("local[*]")
           .config("spark.driver.host","127.0.0.1") 
           .config("spark.driver.bindAddress","127.0.0.1")
           .appName("PySpark_basic")
           .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/14 16:02:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###  스파크 세션 객체를 사용해서 스파크 어플리케애션을 만든다 

In [6]:
spark

In [8]:
spark._sc

<SparkContext master=local[*] appName=PySpark_basic>

In [9]:
spark.sql

<bound method SparkSession.sql of <pyspark.sql.session.SparkSession object at 0x12f030310>>

In [7]:
dir(spark)

['Builder',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activeSession',
 '_conf',
 '_convert_from_pandas',
 '_createFromLocal',
 '_createFromRDD',
 '_create_dataframe',
 '_create_from_pandas_with_arrow',
 '_create_shell_session',
 '_getActiveSessionOrCreate',
 '_get_numpy_record_dtype',
 '_inferSchema',
 '_inferSchemaFromList',
 '_instantiatedSession',
 '_jconf',
 '_jsc',
 '_jsparkSession',
 '_jvm',
 '_repr_html_',
 '_sc',
 'builder',
 'catalog',
 'conf',
 'createDataFrame',
 'getActiveSession',
 'newSession',
 'range',
 'read',
 'readStream',
 'sparkContext',
 'sql',
 'stop',
 'streams',
 'table',
 'udf',
 'version'

## 2. 파이스파크 데이터프레임 생성하기 

- 파이스파크는 트랜스포머와 액션 두 가지의 함수나 메서드를 제공
- 실제 트랜스포머는 지연처리이고 액션이 실행되어야 전체가 실행된다.

## 2-1 스파크에서 제공하는 Row 클래스 사용하기 


## 스파크 데이터프레임(Spark DataFrame) 
- 스파크에서 가장 중요하고 널리 사용되는 데이터 구조 중 하나
- 데이터프레임은 분산 데이터 처리 작업을 수행하기 위한 효과적인 방법을 제공하며, 스파크 SQL 라이브러리의 핵심 구성 요소 

### 스파크 데이터프레임은 다음과 같은 특징

- 구조화된 데이터: 데이터프레임은 행과 열로 구성된 테이블 형태의 구조를 갖습니다. 각 열은 이름을 가지며 데이터 타입을 가리킵니다.

- 비타입(type-safe) API: 스파크 데이터프레임은 비타입 API를 제공하여 데이터 타입을 정적으로 검사하므로 데이터의 일관성을 보장합니다.

- 분산 처리: 스파크 클러스터에서 데이터프레임은 분산 처리를 지원하며, 대용량 데이터를 효율적으로 처리할 수 있습니다.

- 인메모리 처리: 스파크는 데이터를 인메모리로 처리하여 빠른 속도와 성능을 제공합니다.

- 다양한 데이터 소스 지원: 스파크 데이터프레임은 다양한 데이터 소스와 통합되어 있으며, CSV, JSON, Parquet, Avro, JDBC, Apache Hive 등 다양한 데이터 포맷 및 저장소에서 데이터를 읽고 쓸 수 있습니다.

## 스파크(Spark)에서는 데이터 처리 작업을 수행 방식 

- "트랜스포메이션(Transformations)"
- "액션(Actions)"

### 트랜스포메이션(Transformations):

- 트랜스포메이션은 스파크에서 데이터를 변환하고 조작하기 위해 사용되는 연산
- 트랜스포메이션은 데이터프레임, RDD(Resilient Distributed Dataset) 등의 분산 데이터 구조를 기반으로 작동하며, 원본 데이터를 변경하지 않고 새로운 데이터셋을 생성합니다. 
- 트랜스포메이션은 지연 연산(lazy evaluation)을 사용하므로 실행 계획을 구축하고 실제 연산을 수행하기 전까지는 실제로 계산되지 않습니다.

### 액션(Actions):
- 액션은 스파크에서 트랜스포메이션의 결과를 실제로 실행하고 결과를 반환하는 연산입니다. 
- 액션은 지연 연산(lazy evaluation)을 트리거하며, 실제로 데이터를 계산하고 결과를 반환합니다. 
- 액션은 스파크 애플리케이션의 실행을 시작하는 지점이며, 결과를 드라이버 프로그램으로 가져올 때까지 모든 트랜스포메이션을 실행합니다.
- count: 데이터셋의 요소 수를 반환합니다.
- collect: 모든 데이터를 드라이버로 수집합니다. (주의: 대용량 데이터에 사용하면 메모리 문제가 발생할 수 있습니다)
- saveAsTextFile, write: 데이터를 파일로 저장합니다.
- reduce: 데이터셋의 모든 요소를 줄여서 하나의 값으로 결합합니다.
- foreach: 각 요소에 대해 사용자 정의 작업을 수행합니다.



### 스파크의 데이터프레임 생성 

- 주요 생성은 외부 데이터를 포맷에 맞혀 읽어오거나 createDataFrame 으로 생성

In [10]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

### 지연처리도 아직 실행되지 않음 

In [11]:
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

### 스키마 구성은 볼 수 있음

In [12]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



### 실제 데이터프레임 생성을 액션으로 실행하기
- 지연된 모든 연산을 실행한다 

In [13]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



## 2-2 일반적인 파이선 리스트로 파이스파크 데이터프레임 생성 가능 

- 스키마 정보를 문자열로 지정해서 처리
- 'a long, b double, c string, d date, e timestamp' : 변수명과 타입을 쌍으로 처리 

### 출력된 결과를 판다스 처럼 보기 위한 세팅 

In [14]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

### 데이터를 리스트 및 스키마 정보를 별도 부여해서 데이터프레임 생성하기 

In [15]:
df1 = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')


In [16]:
df1

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [17]:
df1.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

